In [18]:
import math
from sklearn import neighbors
import numpy as np
import os
import os.path
import pickle
import cv2
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from sklearn.decomposition import PCA

# /Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Train/X_train.npy
# /Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Train/y_train.npy
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []

    # Load training data
    X_train = np.load(os.path.join(train_dir, 'X_train.npy'))
    y_train = np.load(os.path.join(train_dir, 'y_train.npy'))

    # Loop through each training sample
    for features, label in zip(X_train, y_train):
        # Append features and corresponding label to X and y
        X.append(features)
        y.append(label)

    # Determine how many neighbors to use for weighting in the KNN classifier
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    # Create and train the KNN classifier
    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    # Save the trained KNN classifier
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf


def predict(X_test_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    with open(model_path, 'rb') as f:
        knn_model = pickle.load(f)
    predictions = knn_model.predict(X_test_path)
    return predictions


if __name__ == "__main__":
    # STEP 1: Train the KNN classifier and save it to disk
    # Once the model is trained and saved, you can skip this step next time.
    print("Training KNN classifier...")
    classifier = train("/Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Train", model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Training complete!")

    # Load test data and model
    data = np.load('/Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Test/X_test.npy')
    actual_labels = np.load('/Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/Face_Output/Face_Output_Split_Test/y_test.npy')
    model = "/Users/zhengweing/Desktop/Current/CSCI158/FaceAndIris_Recognition/trained_knn_model.clf"

    # STEP 2: Using the trained classifier, make predictions for unknown images
    predicted_labels = predict(X_test_path=data, model_path=model)

    # Display each data's label and actual class
    for i in range(len(actual_labels)):
        print("Data", i+1, "Predicted Label:", predicted_labels[i], "Actual Label:", actual_labels[i])

    # Calculate accuracy
    correct_predictions = np.sum(predicted_labels == actual_labels)
    total_predictions = len(actual_labels)
    accuracy = correct_predictions / total_predictions
    print("Accuracy:", accuracy)

    cv2.destroyAllWindows()




Training KNN classifier...
Training complete!
Data 1 Predicted Label: S0001_F_30_20_Cropped.jpg Actual Label: S0001_F_30_19_Cropped.jpg
Data 2 Predicted Label: S0016_M_34_2_Cropped.jpg Actual Label: S0009_M_22_19_Cropped.jpg
Data 3 Predicted Label: S0008_F_25_3_Cropped.jpg Actual Label: S0008_F_25_1_Cropped.jpg
Data 4 Predicted Label: S0014_M_54_7_Cropped.jpg Actual Label: S0003_M_32_7_Cropped.jpg
Data 5 Predicted Label: S0004_M_16_18_Cropped.jpg Actual Label: S0005_M_11_1_Cropped.jpg
Data 6 Predicted Label: S0018_F_43_9_Cropped.jpg Actual Label: S0005_M_11_26_Cropped.jpg
Data 7 Predicted Label: S0005_M_11_8_Cropped.jpg Actual Label: S0005_M_11_10_Cropped.jpg
Data 8 Predicted Label: S0015_M_26_11_Cropped.jpg Actual Label: S0018_F_43_4_Cropped.jpg
Data 9 Predicted Label: S0004_M_16_33_Cropped.jpg Actual Label: S0022_M_47_1_Cropped.jpg
Data 10 Predicted Label: S0013_M_34_5_Cropped.jpg Actual Label: S0013_M_34_3_Cropped.jpg
Data 11 Predicted Label: S0009_M_22_24_Cropped.jpg Actual Label: 

ValueError: X has 2 features, but KNeighborsClassifier is expecting 7 features as input.